In [ ]:
# data Ingestion 

In [ ]:
# Entity
from datetime import datetime
import os 
from NetworkSecurity.constants import training_pipeline 

class TrainingPipelineConfig:
    def __init__(self, timestamp=datetime.now()):
        timestamp=timestamp.strftime('%m_%d_%Y_%H_%M_%S')
        

class DataIngestionConfig:
    def __init__(self,training_pipeline_config:):
        pass

train.csv
